In [93]:
%run ../Python_files/util_data_storage_and_load.py

In [111]:
%run ../Python_files/util.py

In [110]:
from gurobipy import *

try:
    # Create a new model
    m = Model("mip1")
    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")
    # Integrate new variables
    m.update()
    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)
    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")
    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")
    m.optimize()
    for v in m.getVars():
        print v.varName, v.x
        print 'Obj:', m.objVal
except GurobiError:
    print 'Error reported'

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [1e+00, 2e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 4e+00]
Found heuristic solution: objective 2
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0%
x 1.0
Obj: 3.0
y 0.0
Obj: 3.0
z 1.0
Obj: 3.0


In [80]:
import numpy as np
from numpy.linalg import inv

x = np.random.rand(12).reshape(3, 4)

In [70]:
x = np.matrix(x)

In [71]:
x

matrix([[ 0.26866657,  0.1480158 ,  0.73638968,  0.38257636],
        [ 0.3097576 ,  0.88383302,  0.61919737,  0.79336855],
        [ 0.19667063,  0.92730648,  0.4690703 ,  0.98224238]])

In [72]:
K = np.size(x, 1)

In [73]:
x_mean = sum(x[:,j] for j in range(K)) 

In [74]:
x_mean

matrix([[ 1.53564842],
        [ 2.60615654],
        [ 2.57528978]])

In [75]:
np.dot(x_mean, np.transpose(x_mean))

matrix([[ 2.35821606,  4.00214017,  3.95473967],
        [ 4.00214017,  6.79205192,  6.71160829],
        [ 3.95473967,  6.71160829,  6.63211743]])

In [88]:
import numpy as np
from numpy.linalg import inv

def inv_samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return inv(S)

In [86]:
y = inv_samp_cov(x)

In [87]:
y

matrix([[ 10.77564106,  11.60001975,  -5.43004836],
        [ 11.60001975,  30.74021677, -18.96731574],
        [ -5.43004836, -18.96731574,  21.07428929]])

In [107]:
from gurobipy import *

def OD_matrix(x, A, l):
    """
    x: sample matrix, each column is a link flow vector sample; m * K
    K: number of samples
    l: length of lam
    A: path-link incidence matrix; m * r
    P: probability matrix, each row sum up to 1; l * r
    lam: OD flow vector; estimation target, l * 1
    ----------------
    Example:
    m = 24
    r = 140
    l = 56
    ----------------
    """
    x = np.matrix(x)
    A = np.matrix(A)
    model = Model("OD_matrix")
    z = model.addVar(obj=1, vtype="C", name="z")
    lam, P = {}, {}
    for i in range(l):
        lam[i] = model.addVar(obj=0, vtype="C", name="lam[%s]"%i)
        for j in range(r):
            P[i,j] = model.addVar(obj=0, vtype="C", name="P[%s,%s]"%(i,j))
    model.update()
    
    # add constraint for probability matrix P
    for i in range(l):
        coef = [1 for j in range(r)]
        var = [P[i,j] for j in range(r)]
        model.addConstr(LinExpr(coef,var), "=", 1)
        for j in range(r):
            model.addConstr(P[i,j], ">", 0)
    
    # add nonnegative constraint for O-D flow vector lam
    for j in range(r):
        model.addConstr(lam[j], ">", 0)
        
    # add constraint for the relationship between z and x, A, P, lam
    x = np.matrix(x)
    K = np.size(x, 1)
    inv_S = inv_samp_cov(x)
    Q = np.transpose(P)
    likelihood = - (1/2) * sum(np.dot(np.dot(np.transpose(x[:,k] - np.transpose(np.dot(A, Q), lam)), \
                                     inv_S), x[:,k] - np.transpose(np.dot(A, Q), lam)) \
                              for k in range(K))
    model.addConstr(likelihood, "=", z)
    
    model.update()
    model.__data = lam, P 
    return model

In [105]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [112]:
link_day_Jan_dict = zload('../temp_files/link_day_Jan_dict.pkz')

In [116]:
week_day_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [118]:
K = len(week_day_list)

In [124]:
x = np.zeros((24, K))

In [ ]:
m = 24

for k in range(K):
    for i in range(m):
        x[i, k] = link_day_Jan_dict['link_' + str(i) + '_' + str(week_day_list[k])].AM_flow

In [141]:
link_day_Jan_dict['link_2_2'].AM_flow, link_day_Jan_dict['link_2_1'].AM_flow

(632510.40783107991, 632510.40783107991)

In [137]:
x

array([[ 341564.65150776,  341564.65150776,  341564.65150776,
         341564.65150776,  341564.65150776,  341564.65150776,
         341564.65150776,  341564.65150776,  341564.65150776,
         341564.65150776,  341564.65150776,  341564.65150776,
         341564.65150776,  341564.65150776,  341564.65150776,
         341564.65150776,  341564.65150776,  341564.65150776,
         341564.65150776,  341564.65150776,  341564.65150776,
         341564.65150776],
       [ 361581.32068239,  361581.32068239,  361581.32068239,
         361581.32068239,  361581.32068239,  361581.32068239,
         361581.32068239,  361581.32068239,  361581.32068239,
         361581.32068239,  361581.32068239,  361581.32068239,
         361581.32068239,  361581.32068239,  361581.32068239,
         361581.32068239,  361581.32068239,  361581.32068239,
         361581.32068239,  361581.32068239,  361581.32068239,
         361581.32068239],
       [ 632510.40783108,  632510.40783108,  632510.40783108,
         632510.

In [115]:
link_day_Jan_dict['link_23_1'].AM_flow

136261.48062420369

In [14]:
n = 200
c, x_pos, y_pos = make_data(n)
m = n
k = 20
model = kmedian(m, n, c, k)


model.optimize()
x,y = model.__data
edges = [(i,j) for (i,j) in x if x[i,j].X == 1]
nodes = [j for j in y if y[j].X == 1]
print "Optimal value=", model.ObjVal
print "Selected nodes:", nodes
print "Edges:", edges

<gurobi.Model MIP instance k-median: 40002 constrs, 40200 vars, Parameter changes: LogFile=gurobi.log>
Optimize a model with 40002 rows, 40200 columns and 80400 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-03, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 2e+01]
Found heuristic solution: objective 0.708945
Presolve removed 39800 rows and 39800 columns
Presolve time: 0.03s
Presolved: 202 rows, 400 columns, 800 nonzeros
Variable types: 0 continuous, 400 integer (400 binary)

Root relaxation: objective 0.000000e+00, 21 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s

Explored 0 nodes (21 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objectiv